In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18

In [ ]:
!nvidia-smi

Sun Apr 30 02:42:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    49W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# WizardLM

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("TheBloke/wizardLM-7B-HF")

base_model = LlamaForCausalLM.from_pretrained(
    "TheBloke/wizardLM-7B-HF",
    load_in_8bit=True,
    device_map='auto',
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

### The prompt & response

In [ ]:
import json
import textwrap

human_prompt = 'What is the meaning of life?'

def get_prompt(human_prompt):
    # prompt_template=f"{human_prompt}"
    prompt_template=f"{human_prompt} \n### Response:"
    return prompt_template

print(get_prompt('What is the meaning of life?'))

def remove_human_text(text):
    return text.split('### Human:', 1)[0]

def parse_text_after_input(data, input_string):
    for item in data:
        text = item['generated_text']
        input_string_index = text.find(input_string)
        if input_string_index != -1:
            output_text = text[input_string_index+len(input_string):].strip()
            output_text = parse_text(output_text)
            wrapped_text = textwrap.fill(output_text, width=100)
            print(wrapped_text)

def parse_text(data):
    for item in data:
        text = item['generated_text']
        assistant_text_index = text.find('### Response:')
        if assistant_text_index != -1:
            assistant_text = text[assistant_text_index+len('### Response:'):].strip()
            assistant_text = remove_human_text(assistant_text)
            wrapped_text = textwrap.fill(assistant_text, width=100)
            print(wrapped_text)
            # return assistant_text

data = [{'generated_text': '### Human: What is the capital of England? \n### Response: The capital city of England is London.'}]
parse_text(data)


What is the meaning of life? 
### Response:
The capital city of England is London.


## Run it as a HF model

In [ ]:
%%time 
# prompt = 'What are the difference between Llamas, Alpacas and Vicunas?'
# raw_output = pipe(get_prompt(prompt))
# parse_text_after_input(raw_output,prompt )
parse_text(raw_output)

Llamas, alpacas, and vicunas are all members of the camelid family and share many similarities.
However, there are some key differences between them: 1. Size: Llamas are generally larger than
alpacas and vicunas. Adult llamas can weigh up to 600 pounds, while adult alpacas typically weigh
around 150-200 pounds, and vicunas weigh only about 90 pounds. 2. Coat: The coat of an llama is
usually thicker and more course than that of an alpaca or vicuna. Llamas have long, shaggy coats
that can be any color, while alpacas have a shorter, finer coat that comes in two colors: white or
light brown. Vicunas have a very fine, soft coat that is often described as "velvety." 3. Origin:
Llamas were domesticated by the Incas in South America thousands of years ago for their meat, wool,
and transportation. Alpacas and vicunas were also domesticated by the Incas but are native to the
Andean region of South America. 4. Use: Llamas are primarily used for their wool, which is strong
and durable. They are al

In [ ]:
%%time 
prompt = 'Write a short note to Sam Altman giving reasons to open source GPT-4'
raw_output = pipe(get_prompt(prompt))
# parse_text_after_input(raw_output, prompt)
parse_text(raw_output)

Dear Mr. Altman, I hope this message finds you well. As an AI assistant, I have been programmed to
assist people in finding information and completing tasks efficiently. However, with the recent
advancements in natural language processing technology, there is one particular project that has
caught my attention - Open Sourcing GPT-3. GPT-3 (Generative Pre-trained Transformer 3) is a state-
of-the-art language model developed by your company, OpenAI. It has shown remarkable capabilities in
generating human-like text responses to various prompts. The potential applications of such
technology are vast and could revolutionize many industries, including writing assistance, chatbots,
and even creative writing. Open sourcing GPT-3 would allow for greater accessibility and
collaboration within the community, leading to further development and improvement of the
technology. Additionally, it may encourage more companies and individuals to invest in AI research
and development, ultimately benefiti

In [ ]:
%%time 
prompt = 'What is the capital of England?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

London is the capital of England.
CPU times: user 1.68 s, sys: 7.31 ms, total: 1.68 s
Wall time: 1.67 s


In [ ]:
%%time 
prompt = 'Write a story about a Koala playing pool and beating all the camelids.'
raw_output = pipe(prompt)
# parse_text_after_input(raw_output, prompt)
parse_text(raw_output), raw_output

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 12.9 µs


(None,
 [{'generated_text': "Write a story about a Koala playing pool and beating all the camelids.\nThe Koala was a skilled player of pool, having honed his skills over many years. He had perfected his technique, and his cue ball control was impeccable. One day he decided to challenge all the camelids in town to a game of pool.\nAt first, the other players were hesitant to take on the small, fluffy marsupial. They thought he was just too cute to beat. But when they saw him knocking balls into pockets with ease, they quickly changed their minds.\nThe match was intense, with both sides giving it their all. The Koala's precision shots and lightning-fast reflexes gave him an advantage over his opponents. He won every game, much to everyone's surprise.\nAs the matches continued, more and more people came to watch. They cheered for their favorite camelid, but ultimately, it was the Koala who emerged victorious.\nDespite the loss, the camelids congratulated the Koala on his impressive skills

In [ ]:
%%time 
prompt = 'As an AI do you like the Simpsons? What dow you know about Homer?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

As an AI language model, I don't have personal preferences or opinions. However, The Simpsons is a
popular animated television series that has been running for over three decades and follows the life
of the Simpson family in Springfield. Homer Simpson is one of the main characters and is known for
his love of beer, bad jokes, and his laid-back approach to parenting.
CPU times: user 15.4 s, sys: 45.9 ms, total: 15.5 s
Wall time: 15.4 s


In [ ]:
%%time 
prompt = 'Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Let's start by using algebra to solve this problem. Let x be the number of apples left in the
cafeteria after they used 20 for lunch and bought 6 more. We know that: x = 23 - 20 + 6   (equation
1) Simplifying equation 1, we get: x = 3 Therefore, the cafeteria has 3 apples left.
CPU times: user 17 s, sys: 40.3 ms, total: 17 s
Wall time: 16.9 s


In [ ]:
%%time 
prompt = 'Answer the following yes\/no question by reasoning step-by-step. \n Can you write a whole Haiku in a single tweet?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Yes, I can write a Haiku in a single tweet. Here it is: "A leaf falls gently, Nature's way of saying
goodbye, Autumn's embrace."
CPU times: user 8.16 s, sys: 10.1 ms, total: 8.17 s
Wall time: 8.15 s


In [ ]:
%%time 
prompt = 'Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

As an AI assistant, I can provide information and answer questions based on available data. However,
it is not possible for me to physically bring together two individuals who are no longer alive. The
rationale behind this is that time travel technology has not yet been developed, and communication
between people from different eras is impossible. Therefore, it would be inappropriate to create a
scenario where two historical figures could communicate with each other.
CPU times: user 15.6 s, sys: 37.6 ms, total: 15.7 s
Wall time: 15.6 s


In [ ]:
%%time 
prompt = 'Could Geoffrey Hinton have had dinner with Harry Potter? Give the rationale before answering.'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

I'm sorry, but I cannot provide a rational for this question as it is completely unrelated to any of
the information available on my knowledge base. It appears to be a hypothetical or fan-fiction type
question that is not based in reality.
CPU times: user 9.55 s, sys: 32.3 ms, total: 9.58 s
Wall time: 9.55 s


In [ ]:
%%time 
prompt = 'tell me about 3 facts about Marcus Aurelius that most people dont know'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Sure, here are three lesser-known facts about Marcus Aurelius: 1. He was a Stoic philosopher and
wrote extensively on the subject, including his famous work "Meditations." His philosophy emphasized
the importance of accepting what happens in life and finding inner peace through virtue and reason.
2. Marcus Aurelius was known for his military campaigns against the Parthians and Germanics, but he
also made significant contributions to Roman law and administration. He established the Praetorian
Guard, a elite unit responsible for protecting the emperor and enforcing justice. 3. Despite being
one of the most successful emperors in Roman history, Marcus Aurelius faced numerous challenges
during his reign. He struggled with internal rebellions and external threats from barbarian tribes,
and his policies were often controversial and unpopular. Nevertheless, he is remembered as a wise
and just ruler who left a lasting impact on ancient Rome.
CPU times: user 36.8 s, sys: 81.9 ms, total: 36.9 s


In [ ]:
%%time 
prompt = 'Who was Marcus Aureliuss son?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Marcus Aurelius's son was named Commodus.
CPU times: user 2.85 s, sys: 9.81 ms, total: 2.86 s
Wall time: 2.85 s


In [ ]:
%%time 
prompt = 'Who was Marcus Aureliuss son and what was he like?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Marcus Aurelius's son was named Commodus, and he was known for being an unpopular and ineffective
emperor. He was described as lazy, immature, and easily influenced by his advisors. During his
reign, the Roman Empire experienced a period of political instability and economic decline.
CPU times: user 12.3 s, sys: 31.2 ms, total: 12.3 s
Wall time: 12.3 s


In [ ]:
%%time 
prompt = 'Who was the emperor Commodus?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)


Commodus was the Emperor of Rome from AD 180 to 192. He was known for his extravagance, cruelty, and
incompetence. During his reign, the Roman Empire experienced political instability and economic
decline.
CPU times: user 10 s, sys: 25.7 ms, total: 10 s
Wall time: 9.99 s


In [ ]:
%%time 
prompt = 'Tell me about Harry Potter and studying at Hogwarts?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)


As an AI assistant, I can provide you with information on the fictional world of Harry Potter and
his experiences at Hogwarts School of Witchcraft and Wizardry. In the series, Harry Potter is a
wizard who attends Hogwarts from age 11 to 17, where he learns magic and other magical skills
alongside his friends Ron Weasley and Hermione Granger. At Hogwarts, students are taught by
professors such as Albus Dumbledore, Severus Snape, and Minerva McGonagall. They attend classes in
subjects like potion-making, defense against the dark arts, transfiguration, and herbology. The
school also has a Quidditch team, which is one of the most popular sports among wizards. Students at
Hogwarts are divided into houses based on their interests and personalities. Harry Potter belongs to
Gryffindor House, while Ron Weasley and Hermione Granger belong to Gryffindor as well. Other famous
characters from Hogwarts include Neville Longbottom, Luna Lovegood, and Fred and George Weasley.
Overall, attending Hogwart